Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/manymodels/03_Forecasting/03_Forecasting_Pipeline.png)

# Real-time Forecasting Webservice Deployment - Automated ML
---

In this notebook we deploy multiple webservices to forecast sales in real-time with the models we trained in the last step.

Models are grouped based on their tags and each group is deployed together to the same webservice. You can customize your grouping strategy by simply playing with the model tags. 

### Prerequisites 
At this point, you should have already: 
1. Created your AML Workspace using the [00_Setup_AML_Workspace notebook](../../00_Setup_AML_Workspace.ipynb)
2. Run [01_Data_Preparation.ipynb](../../01_Data_Preparation.ipynb) to create the dataset
3. Run [02_AutoML_Training_Pipeline.ipynb](../02_AutoML_Training_Pipeline/02_AutoML_Training_Pipeline.ipynb) to train the models

## 1.0 Connect to workspace

In [1]:
import azureml.core
from azureml.core import Workspace, Datastore
import pandas as pd
import os
import sys

sys.path.append("../../../")


In [2]:
from ml_service.utils.pipeline_info import Info
from ml_service.utils.env_variables import Env
from ml_service.utils.aml_workspace import Connect


# set up workspace
# ws=Workspace.from_config()
# Get the variables defined in config file .env
e=Env()

ws = Connect().authenticate()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EMZLLB2TH to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...


Failed to authenticate to tenant '491d83df-1091-40f8-bcf9-b112f9a35fcf' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS53003: Access has been blocked by Conditional Access policies. The access policy does not allow token issuance.\r\nTrace ID: 0911c1ea-0605-4a62-b03e-d6149bcbe500\r\nCorrelation ID: df89077a-1f21-46be-b0c7-1375f7551e45\r\nTimestamp: 2021-10-07 17:38:40Z","error_codes":[53003],"timestamp":"2021-10-07 17:38:40Z","trace_id":"0911c1ea-0605-4a62-b03e-d6149bcbe500","correlation_id":"df89077a-1f21-46be-b0c7-1375f7551e45","error_uri":"https://login.microsoftonline.com/error?code=53003","suberror":"message_only"}'.Will continue to look for other tenants to find subscriptions to which you have access
Failed to authenticate to tenant '79c07380-cc98-41bd-806b-0ae925588f66' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_descr

Interactive authentication successfully completed.


In [3]:
# Take a look at Workspace
ws.get_details()

# set up datastores
dstore = Datastore.get(ws, e.datastore_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Default datastore name'] = dstore.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])

## 2.0 Get models to be deployed

### 2.1 Get models registered in the workspace that had been trained by a run

In [4]:
from azureml.core import Model

info = Info(ws, e.experiment_name, os.environ.get("BUILDID"))

# runid = info.get_run_id()   #'aa417fa2-45ca-4060-9026-adb81b164c8a' # update the pipeline run 
runid='6dc0c1e6-3335-4278-b355-d959e9ab70ac'
tags = [['ModelType', 'AutoML'], ['RunId', runid]]

models = Model.list(ws, tags=tags, latest=True, expand=False)
print('Got '+str(len(models))+' models from the workspace.')

Got 10 models from the workspace.


### 2.2 Group models by store

We will group the models by store. Therefore, each group will contain three models, one for each of the orange juice brands, and all of them corresponding to the same store.

You can change the grouping strategy by modifying the `grouping_tags` variable below and specifying the names of the tags you want to use for grouping. For convenience, we have created two additional grouping tags you can use:
- `StoreGroup10`: groups stores 10 by 10
- `StoreGroup100`: groups stores 100 by 100

To create custom tags, modify the `tags_dict` object in the [training script](scripts/train.py) and run the training again.

In [5]:
grouping_tags = ['Store']

In [6]:
grouped_models = {}
for m in models:
    
    if m.tags['ModelType'] == '_meta_':
        continue
    
    group_name = '/'.join([m.tags[t] for t in grouping_tags])
    group = grouped_models.setdefault(group_name, [])
    group.append(m)

## 3.0 Configure deployment

### 3.1 Define inference environment

In [7]:
from scripts.helper import get_automl_environment
forecast_env = get_automl_environment(workspace=ws, training_pipeline_run_id=runid, training_experiment_name='manymodels-training-pipeline')

### 3.2 Define inference configuration

In [8]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(
    entry_script='forecast_webservice.py',
    source_directory='./scripts',
    environment=forecast_env
)

### 3.3 [Option A] Define deploy configuration using ACI (dev/test)

Use this option to deploy the models to Azure Container Instances, indicated for dev/test environments.

In [9]:
from azureml.core.webservice import AciWebservice

deployment_type = 'aci'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
deployment_target = None

### 3.3 [Option B] Define deploy configuration using AKS (production)

Use this option to deploy the models to Azure Kubernetes Services, indicated for production environments.

In [10]:
aks_target_name = 'codingforge-aks'  # aks with TLS enabled
# aks_target_name = 'coding-forge-aks'
aks_resource_name = aks_target_name
aks_resource_group = ws.resource_group

In [11]:
from azureml.core.compute import AksCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    aks_target = AksCompute(ws, aks_target_name)
    aks_target.provisioning_configuration().enable_ssl(leaf_domain_label="codingforge", overwrite_existing_domain=True)
    #provisioning_config = AksCompute.provisioning_configuration()
    #provisioning_config.enable_ssl(leaf_domain_label="coding-forge")

    #found_aks_cluster = True
    print('AKS cluster already attached. Skip the optional step below and jump to "Configure AKS"')
except ComputeTargetException:
    print('AKS cluster not attached yet. Attempting to attach compute now')
    attach_config = AksCompute.attach_configuration(
        resource_group=aks_resource_group,
        cluster_name=aks_resource_name
    )

    aks_target = ComputeTarget.attach(ws, aks_target_name, attach_config)
    aks_target.wait_for_completion(show_output=True)    

AKS cluster already attached. Skip the optional step below and jump to "Configure AKS"


#### Configure AKS

In [12]:
from azureml.core.webservice import AksWebservice

deployment_type = 'aks'
deployment_config = AksWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
deployment_target = aks_target

## 4.0 Deploy the models

We will now deploy one webservice for each of the groups of models. Deployment takes some minutes to complete, so we'll request all of them and then wait for them to finish.

In [29]:
deployments = []
for group_name, group_models in grouped_models.items():
    
    service_name = '{prefix}manymodels-{group}'.format(
        prefix='test-' if deployment_type == 'aci' else '',
        group=group_name
    ).lower()
    
    print('Launching deployment of {}...'.format(service_name))
    service = Model.deploy(
        workspace=ws,
        name=service_name,
        models=group_models,
        inference_config=inference_config,
        deployment_config=deployment_config,
        deployment_target=deployment_target,
        overwrite=True
    )
    print('Deployment of {} started'.format(service_name))
    
    deployments.append({ 'service': service, 'group': group_name, 'models': group_models })
    

Launching deployment of manymodels-1001...
Deployment of manymodels-1001 started
Launching deployment of manymodels-1000...
Deployment of manymodels-1000 started
Launching deployment of manymodels-1002...
Deployment of manymodels-1002 started
Launching deployment of manymodels-1003...
Deployment of manymodels-1003 started


In [30]:
models_deployed = {}
for deployment in deployments:
    
    service = deployment['service']
    print('Waiting for deployment of {} to finish...'.format(service.name))
    service.wait_for_deployment(show_output=True)
    if service.state != 'Healthy':
        print('DEPLOYMENT FAILED FOR SERVICE {}'.format(service.name))
    
    service_info = {
        'webservice': service.name,
        'state': service.state,
        'endpoint': service.scoring_uri if service.state == 'Healthy' else None,
        'key': service.get_keys()[0] if service.auth_enabled and service.state == 'Healthy' else None
    }

    print("Rolling out the deployment for service {}".format(service.name))
    # Store deployment info for each deployed model
    for m in deployment['models']:
        models_deployed[m.name] = {
            'version': m.version,
            'group': deployment['group'],
            **service_info
        }


Waiting for deployment of manymodels-1001 to finish...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-07 13:52:11-04:00 Creating Container Registry if not exists.
2021-10-07 13:52:11-04:00 Registering the environment.
2021-10-07 13:52:12-04:00 Use the existing image.
2021-10-07 13:52:13-04:00 Creating resources in AKS.
2021-10-07 13:52:14-04:00 Submitting deployment to compute.
2021-10-07 13:52:14-04:00 Checking the status of deployment manymodels-1001..
2021-10-07 13:53:03-04:00 Checking the status of inference endpoint manymodels-1001.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Rolling out the deployment for service manymodels-1001
Waiting for deployment of manymodels-1000 to finish...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-local

### 4.2 Test the webservices

We can query for multiple models into the same request, but all of them need to be from the same store, as each endpoint only contains models corresponding to one particular store.

In [31]:
from azureml.core import Datastore

# Please change the following to point to your own blob container and pass in account_key
blob_datastore_name =   e.blob_datastore_name           # "automl_many_models"
container_name =        e.container_name                # "automl-sample-notebook-data"
account_name =          e.account_name                  # "automlsamplenotebookdata"

oj_datastore = Datastore.register_azure_blob_container(workspace=ws, 
                                                       datastore_name=blob_datastore_name, 
                                                       container_name=container_name,
                                                       account_name=account_name,
                                                       create_if_not_exists=True)

In [32]:
from azureml.core.dataset import Dataset
inference_name_small = 'oj_inference_small'

inference_ds_small = Dataset.Tabular.from_delimited_files(path=oj_datastore.path(inference_name_small + '/'), validate=False)
all_df = inference_ds_small.to_pandas_dataframe()

In [33]:
from scripts.helper import get_model_name
store = 1002
brand = 'dominicks'
tags_dict = {'store':store, 'brand': brand}
model_name = get_model_name(tags_dict)

In [34]:
dominicks_test_data = all_df.loc[(all_df['Store']==store) & (all_df['Brand']==brand)]
print(dominicks_test_data.head(5))

    WeekStarting  Store      Brand  Quantity  Advert  Price   Revenue
108 1992-06-04    1002   dominicks  11957     1       2.07   24750.99
109 1992-06-11    1002   dominicks  16608     1       2.49   41353.92
110 1992-06-18    1002   dominicks  15073     1       2.63   39641.99
111 1992-06-25    1002   dominicks  10881     1       2.16   23502.96
112 1992-07-02    1002   dominicks  9384      1       2.66   24961.44


In [35]:
dominicks_test_data_json = dominicks_test_data[:].to_json(orient='records', date_format='iso')

In [36]:
test_data = [{
        "group_column_names": ['Store', 'Brand'], # This is the same list that is passed in the training script
        "time_column_name": "WeekStarting", # This is the same value for time_column_name that is passed in the training script
        "data": dominicks_test_data_json
    }]

In [37]:
test_data

[{'group_column_names': ['Store', 'Brand'],
  'time_column_name': 'WeekStarting',
  'data': '[{"WeekStarting":"1992-06-04T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":11957,"Advert":1,"Price":2.07,"Revenue":24750.99},{"WeekStarting":"1992-06-11T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":16608,"Advert":1,"Price":2.49,"Revenue":41353.92},{"WeekStarting":"1992-06-18T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":15073,"Advert":1,"Price":2.63,"Revenue":39641.99},{"WeekStarting":"1992-06-25T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":10881,"Advert":1,"Price":2.16,"Revenue":23502.96},{"WeekStarting":"1992-07-02T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":9384,"Advert":1,"Price":2.66,"Revenue":24961.44},{"WeekStarting":"1992-07-09T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":9029,"Advert":1,"Price":1.92,"Revenue":17335.68},{"WeekStarting":"1992-07-16T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity"

In [38]:
import requests
import json

try:
    url = models_deployed[model_name]['endpoint']
    key = models_deployed[model_name]['key']    
except KeyError as e:
    raise ValueError(f'Model for store {store} and brand {brand} has not been deployed')

request_headers = {'Content-Type': 'application/json'}
if key:
    request_headers['Authorization'] = f'Bearer {key}'

response = requests.post(url, json=test_data, headers=request_headers)
# response.json() # uncomment the line to see the values returned in the response

## 5.0 Group all models into a single routing endpoint

We can now group all the services into a single entry point, so that we don't have to handle each endpoint separately. 
For that, we'll register the `endpoints` object as a model, and deploy it as a webservice. This webservice will receive the incoming requests and route them to the appropiate model service, acting as the unique entry point for outside requests.

### 5.1 Register endpoints dict as an AML model

In [39]:
import joblib

joblib.dump(models_deployed, 'models_deployed.pkl')

dep_model = Model.register(
    workspace=ws, 
    model_path ='models_deployed.pkl', 
    model_name='deployed_models_info',
    tags={'ModelType': '_meta_'},
    description='Dictionary of the service endpoint where each model is deployed'
)

Registering model deployed_models_info


### 5.2 Deploy routing webservice

In [40]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
routing_env = Environment(name="many_models_routing_environment")
routing_env_deps = CondaDependencies.create(pip_packages=['azureml-defaults', 'joblib', 'pandas'])
routing_env.python.conda_dependencies = routing_env_deps

routing_infconfig = InferenceConfig(
    entry_script='routing_webservice.py',
    source_directory='./scripts',
    environment=routing_env
)

# Reuse deployment config with lower capacity
deployment_config.cpu_cores = 0.1
deployment_config.memory_gb = 0.5

routing_service = Model.deploy(
    workspace=ws,
    name='routing-manymodels',
    models=[dep_model],
    inference_config=routing_infconfig,
    deployment_config=deployment_config,
    deployment_target=deployment_target,
    overwrite=True
)
routing_service.wait_for_deployment(show_output=True)

assert routing_service.state == 'Healthy'

print('Routing endpoint deployed with URL: {}'.format(routing_service.scoring_uri))

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-10-07 13:53:35-04:00 Creating Container Registry if not exists.
2021-10-07 13:53:35-04:00 Registering the environment.
2021-10-07 13:53:36-04:00 Use the existing image.
2021-10-07 13:53:37-04:00 Creating resources in AKS.
2021-10-07 13:53:38-04:00 Submitting deployment to compute.
2021-10-07 13:53:38-04:00 Checking the status of deployment routing-manymodels..
2021-10-07 13:54:27-04:00 Checking the status of inference endpoint routing-manymodels.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Routing endpoint deployed with URL: https://codingforgeeupail.eastus.cloudapp.azure.com:443/api/v1/service/routing-manymodels/score


### 5.3 Test the webservice

This new endpoint can be called with data from different stores or brands, and it will automatically route the request to the appropiate model endpoint.

In [41]:
import requests
import json
url = routing_service.scoring_uri

request_headers = {'Content-Type': 'application/json'}
if routing_service.auth_enabled:
    keys = routing_service.get_keys()
    request_headers['Authorization'] = 'Bearer {}'.format(keys[0])

response = requests.post(url, json=test_data, headers=request_headers)
# response.json() # uncomment to show the response

In [42]:
response.json()

['[{"WeekStarting":"1992-06-04T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":11957,"Advert":1,"Price":2.07,"Revenue":24750.99,"Predictions":14623.2254901961},{"WeekStarting":"1992-06-11T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":16608,"Advert":1,"Price":2.49,"Revenue":41353.92,"Predictions":14623.2254901961},{"WeekStarting":"1992-06-18T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":15073,"Advert":1,"Price":2.63,"Revenue":39641.99,"Predictions":14623.2254901961},{"WeekStarting":"1992-06-25T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":10881,"Advert":1,"Price":2.16,"Revenue":23502.96,"Predictions":14623.2254901961},{"WeekStarting":"1992-07-02T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":9384,"Advert":1,"Price":2.66,"Revenue":24961.44,"Predictions":14623.2254901961},{"WeekStarting":"1992-07-09T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":9029,"Advert":1,"Price":1.92,"Revenue":17335.68,"Predictions":14623.225

In [43]:
display(test_data)
display(url)
display(keys)
display(request_headers)

[{'group_column_names': ['Store', 'Brand'],
  'time_column_name': 'WeekStarting',
  'data': '[{"WeekStarting":"1992-06-04T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":11957,"Advert":1,"Price":2.07,"Revenue":24750.99},{"WeekStarting":"1992-06-11T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":16608,"Advert":1,"Price":2.49,"Revenue":41353.92},{"WeekStarting":"1992-06-18T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":15073,"Advert":1,"Price":2.63,"Revenue":39641.99},{"WeekStarting":"1992-06-25T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":10881,"Advert":1,"Price":2.16,"Revenue":23502.96},{"WeekStarting":"1992-07-02T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":9384,"Advert":1,"Price":2.66,"Revenue":24961.44},{"WeekStarting":"1992-07-09T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity":9029,"Advert":1,"Price":1.92,"Revenue":17335.68},{"WeekStarting":"1992-07-16T00:00:00.000Z","Store":1002,"Brand":"dominicks","Quantity"

'https://codingforgeeupail.eastus.cloudapp.azure.com:443/api/v1/service/routing-manymodels/score'

('', '')

{'Content-Type': 'application/json',
 'Authorization': 'Bearer '}

## Clean up after running the example

When you complete the tutorial you can remove all webservices create for the many models solution

In [44]:
#from azureml.core import Webservice
#for webservice in Webservice.list(ws):
#   print('name:', webservice.name)
#   if "manymodels" in webservice.name:
#       Webservice(ws, name = webservice.name).delete()